Process dr3 lightcurves produced by the IPAC Forced photometry pipeline, make quick fits. 

In [ ]:
import sys, os

In [ ]:
from ampel.secret.AmpelVault import AmpelVault
from ampel.secret.DictSecretProvider import DictSecretProvider
from ampel.dev.DevAmpelContext import DevAmpelContext
from ampel.alert.load import DirAlertLoader
from ampel.alert.load import FileAlertLoader


In [ ]:
AMPEL_CONF = '/home/jnordin/github/ampelJan25/Ampel-HU-astro/ampel_conf.yaml'

In [ ]:
# Select date of query with format e.g. '20210610'
channel = 'lcana'
ITER_MAX = 10
ALERT_DIR = '/home/jnordin/github/ampelMay24/Ampel-HU-astro/notebooks/dr3_bts'

In [ ]:
ctx = DevAmpelContext.load(
    config = AMPEL_CONF,
    db_prefix = 'bts_bcorr',
    purge_db = False,
#    vault = vault
)
ctx.add_channel(
    name=channel,
    access=['ZTF', 'ZTF_PUB', 'ZTF_PRIV']
)

In [ ]:
directives = [
        {
            "channel": channel,
            "ingest": {
                "mux": {
                    "unit": "ZiMongoMuxer",
                    "config": {
                        "db_complete": True,     
                    },
                    "combine": [
                        {
                            "unit": "ZiT1Combiner",
                            "state_t2": [
                                {
                                    "unit": "T2TabulatorRiseDecline",
                                    "config": {
                                        'tabulator': [ {'unit': 'ZTFFPTabulator'} ],    
                                        't_cadence': 3,
                                        'significant_bands': ['ztfg', 'ztfr', 'ztfi'],
                                    }
                                },
                            ]
                        }
                    ],
                }
            }
        }
    ]

In [ ]:
### Update templates such that T2Compiler does not break

In [ ]:
from ampel.util.template import apply_templates
from ampel.log.AmpelLogger import AmpelLogger

In [ ]:
templates = ('resolve_run_time_aliases', 'hash_t2_config')

In [ ]:
apply_templates(ctx, templates, {"directives":directives}, logger=AmpelLogger.get_logger())

In [ ]:
ac = ctx.new_context_unit(
    unit = "AlertConsumer",
    process_name = "AP_test",
    iter_max = ITER_MAX,
    log_profile = os.environ.get('log_profile', 'debug'),
    shaper = "ZiDataPointShaper",
    # Do not use the standard ZTF compiler options as these assume that the stock->name conversion works.
    # Should develop a new id translater which, possibly, use ra,dec to search for the correct name.
    #compiler_opts = "ZiCompilerOptions",
    supplier = {
        "unit": "ZTFIPACForcedPhotometryAlertSupplier",
        'config': {
#            'deserialize': None,
            'alert_history': False,
            'save_file_dir': '/home/jnordin/tmp/baselinecorr',
            'name_file': '/home/jnordin/data/ztf_ipacfp/bts_explorer_241122.csv',
            'file_keys': {
                "id": "ZTFID",
                "ra": "RA",
                "dec": "Dec",
                "raunit": "hourangle",
                "decunit": "deg",
            },
            'loader': {
                'unit': 'DirFileNamesLoader',                
                'config': {'folder': ALERT_DIR, 'extension' : 'txt'}
            }
        }
    },
    directives = directives
)


In [ ]:
n = ac.run()

In [ ]:
print(f"{n} alerts processed for channel {channel}")

In [ ]:
t2w = ctx.new_context_unit(
    unit = 'T2Worker',
    process_name = 'T2Processor_test',
    log_profile = os.environ.get('log_profile', 'default')
)

In [ ]:
t2w.run()